In [ ]:
#import AI Gym Environment 
from google.colab import drive 
drive.mount('/content/gdrive/', force_remount=True)
%cd "gdrive/MyDrive/RLResearch"
!pip install import-ipynb 
import import_ipynb 
pip install xlrd


In [ ]:
import random 
import openpyxl 
from Baseball_Schedule_AIGym import MLBSchedulingEnv 

In [ ]:
class LeagueScheduler(): 

  def __init__(self, league_name):
    #sets up initial constraints to the RL agent 
    #randomly generated only once 
    
    #check the first schedule returned by RL agent 
    #generate MST pseudo-randomly 
    self.mst = 0.5 
    self.teamsSatisfied = 0 #will be evaluated later 
    self.league_name = league_name 
    self.constraints = {}
    
    #set up initial constraints 
    #initial request sent to RL agenst 
    scheduling_env = MLBSchedulingEnv()
    
    #check intiial request via checkReturnedSchedule 
    scheduling_env.render()

    checkReturnedSchedule() 
    self.division = {}
    #init division for the given league_name 
    if self.league_name = 'East':
      self.division = {'Orioles':'Baltimore','Braves':'Atlanta','RedSox':'Boston',
                          'Marlins':'Miami','Yankees':'New York','Mets':'New York',
                          'Rays':'Tampa Bay','Phillies':'Philadelphia','BlueJays':'Toronto',
                          'Nationals':'Washington'}
    elif self.league_name = 'West': 
      self.division = {'Astros':'Houston','Angels':'Los Angeles','Athletics':'Oakland',
                          'Mariners':'Seattle','Rangers':'Texas','Diamondbacks':'Arizona',
                          'Rockies':'Colorado','Dodgers':'Los Angeles','Padres':'San Diego'} 
    else:
      self.division = {'WhiteSox':'Chicago','Guardians':'Cleveland','Tigers':'Detroit',
                              'Royals':'Kansas City','Twins':'Minnosota','Clubs':'Chicago',
                              'Reds':'Cincinnati','Brewers':'Milwaukee', 'Pirates':'Pittsburgh',
                              'Cardinals':'St.Louis'}


    
  
  #generate MST is called when the returned schedule in unsatisfied 
  def generateMST(): 
    #pseduo-randomly generated MST 
    #for MST 
    
    #MST not satified. 
    #if MST < 0.675: increment 0.05 from the past MST 
    if self.mst < 0.675: #systemtaic increment 
      self.mst += 0.05
    else: #greater than equal 
      #select range by random - ranging from 0.01 to 0.05
      range = random.uniform(0.01,0.05) 
      self.mst += range 

    #submit new schedule with this mst 
    submitRandomRequest()

  def submitRandomRequest(): 
    #submits new and/or modified requests at random 
    #call RL agent for a new schedule 
  
    #call checkReturnedSchedule to check the returned MST 
    scheduling_env.modifyMST(self.mst)
    file_name = scheduling_env.render() #assume that the filename was returned from the agent
    evaluateReturnedSchedule(file_name)

  def evaluateReturnedSchedule(file_name): 

    #check each row, check if the team is satisfied 
    #nested dictionary 
    #path to the generated excel file will be updated later 
    excel_location = ("path of file")

    excel = openpyxl.load_workbook(excel_location)
    teams = {}

    #example: teams = {WhiteSox: {series# : (# away_games , # home_games, #interdivisonal games, #intradivisional games )}}
    for team in self.division: 
      #generate nested dictionary 
      teams[team] = {}
      
    sheet = excel.active 
    
    #extract constraints info 
    #probably need to change min_row from here 
    for row in sheet.iter_rows(min_row = 6, min_col = 2, max_row = 10, max_col = 2): 
      for cell in row: #each cell is a constraint
        constraint = cell.value 
        if ':' in constraint: 
          const = constraint.split(':') #split based on : 
          self.constraints[const[0]] = int(const[1])
        else: #not associated with number 
          self.constraints[constraint] = ''
          
    #extract rows containing information about series, away_team, home_team, play or not
    for row in sheet.iter_rows(min_row = 13, min_col=1, max_row= 45252, max_col =4): 
      i = 0 
      series = 0
      away = ''
      home = ''
      play = False 
      for cell in row: 
        if i == 0:
          #series num 
          series = int(cell.value)
        elif i == 1: 
          #away_team
          away = cell.value
        elif i == 2: 
          #home_team 
          home = cell.value 
        else: # i ==3 
          if cell.value == 'play': 
            play = True 
        
        i += 1 
      #update dictionary accordingly
      
      if play: #there is a game
        #need to update #home games, #away, #interdiv, #intradiv 
        #teams[team] already created 
        
        #if first series encountered
        intradiv = False sssss
        interdiv = False 
        if away in teams and series not in teams[team]: 
          teams[away][series] = (0,0,0,0)
        else: #already encountered 
          tup = teams[away][series]
          if away in teams and home in teams: #intra-div 
            teams[away][series] = (tup[0], tup[1], tup[2], tup[3]+1)
            intradiv = True 
          elif (away in teams and home not in teams) or (home in teams and away not in teams): #inter-div
            teams[away][series] = (tup[0], tup[1], tup[2]+1, tup[3])
            interdiv = True
          #update #of away games 
          tup = teams[away][series]
          teams[away][series] = (tup[0]+1, tup[1], tup[2], tup[3]+1)


        if home in teams and series not in teams[team]: 
          teams[home][series] = (0,0,0,0)
        else: #increment existing tuple
          tup = teams[home][series]
          if intradiv: 
            teams[home][series] = (tup[0], tup[1], tup[2], tup[3]+1)
          elif interdiv: 
            teams[home][series] = (tup[0], tup[1], tup[2]+1, tup[3])
          #update #of home game
          tup = teams[home][series]
          teams[home][series] = (tup[0], tup[1]+1, tup[2], tup[3]+1)

        
      constraintsSatisfied(constraints, teams)

      if self.teamsSatisfied / 30 > self.mst: #satisfied, so terminate 
        return 
      else: #need to request again 
        generateMST() #pseduo-generate MST 

    #after reading all rows from excel, we need to check how many teams are satisfied 
    def constraintsSatisfied(constraints, teams): 
      #check if constraints are satisfied by traversing teams dictionary, which contains information about #of home games, #of away games played in each series 
    
      #define satisfaction: if all constraints are satisfied 
      #again, create a dictionary of teams
      #{WhiteSox: 2, BlueJays: 3} number indicating #of constrains satisfied
      minHomeGames = 0 
      numInterDivisionalGames = 0 
      numsIntraDivisionalGames = 0  
      teamsSatisfied = {}

      numConstraints = len(self.constraints)
      satisfiedConstraints = numConstraints #assume that every constrains are satisfied 
      #init teamsSatisfied dictionary with satisfiedConstraints
      for team in self.division.keys(): 
        teamsSatisfied[team] = satisfiedConstraints

      if 'setNumHomeGamesMinimum' in self.constraints: #check 
        minHomeGames = self.constraints['setNumHomeGamesMinimum']
      if 'numInterDivisionalGames' in self.constraints: 
        numInterDivisionalGames = self.constraints['numInterDivisionalGames']
      if 'numsIntraDivisionalGames' in self.constraints: 
        numIntraDivisionalGames = self.constraints['numsIntraDivisionalGames']
    

      for team, schedule in teams.iteams(): 
        for series, seriesInfo in schedule.items():
          #each has #homegame, #away game, #interdiv, #intradiv 
          if minHomeGames != 0 and seriesInfo[1] < minHomeGames: #didn't satisfy constraint 
            teamsSatisfied[team] -= 1 #decrement satisfied constraints 
          #total games played in a series 
          total_games = seriessInfo[0] + seriesInfo[1] #check for played in every slot 
          if numInterDivisionalGames != 0 and seriesInfo[2] < numInterDivisionalGames:
            teamsSatisfied[team] -= 1
          if numIntraDivisionalGames != 0 and seriesInfo[3] < numIntraDivisionalGames:
            teamsSatisfied[team] -= 1
        
      #count for teamsSatisfied 
      for numSatisfiedConstraints in teamsSatisfied.values():
        if numSatisfiedConstraints == numConstraints:
          self.teamsSatisfied += 1 
      
      #calculate mst 
      if self.teamsSatisfied / 30 < self.mst: #create another random request 
        generateMST()
      else: 
        return #terminate requests
        


In [ ]:
#intiate for each three divisions 
#initate same class, but with different divisions 
LeagueScheduler('West')
LeagueScheduler('East')
LeagueScheduler('Central')